# Config Loader Redaction Walkthrough

This notebook mirrors the helper functions `_normalize_path` and `_redact` from `backtester/core/config_loader.py`.
It demonstrates how secret paths are normalised and how a manifest-safe copy of the configuration is produced.


In [12]:
from collections.abc import Iterable, Mapping, MutableMapping, Sequence
from copy import deepcopy
from typing import Any

REDACTION_TOKEN = "***REDACTED***"


def _normalize_path(path: str | Sequence[str]) -> tuple[str, ...]:
    """
    Turn a secret selector into a canonical tuple of path segments.
    Accepts a dot-path string (e.g. "secrets.api_key") or an iterable of
    segments (e.g. ("secrets", "api_key")).
    Empty segments are ignored to keep the output clean.
    """
    if isinstance(path, str):
        segments = [segment.strip() for segment in path.split(".")]
    else:
        segments = [str(segment).strip() for segment in path]

    normalized = tuple(segment for segment in segments if segment)
    if not normalized:
        raise ValueError("Secret path must contain at least one non-empty segment")
    return normalized


def _redact(
    cfg: Mapping[str, Any],
    secret_keys: Iterable[str | Sequence[str]],
    *,
    token: str = REDACTION_TOKEN,
) -> tuple[Mapping[str, Any], int]:
    """
    Produce a deep-copied version of cfg where the requested secret fields are
    replaced with the redaction token. The function returns the redacted config
    and the count of fields that were scrubbed.
    """
    normalized_paths = [_normalize_path(key) for key in secret_keys]
    print("Normalized paths: ", normalized_paths)
    redacted_cfg: MutableMapping[str, Any] = dict(deepcopy(cfg))
    redacted_count = 0

    for path in normalized_paths:
        print("traversing path:", path)
        current: MutableMapping[str, Any] | None = redacted_cfg

        for segment in path[:-1]:
            print("traversing segment: ", segment)
            if current is None or segment not in current:
                current = None
                print(
                    "current is None or segment not in current",
                    "segment: ",
                    segment,
                    "current",
                    current,
                )
                break

            next_node = current[segment]
            print("next_node: (current[segment])", next_node)
            if isinstance(next_node, MutableMapping):
                if not isinstance(next_node, dict):
                    next_node = dict(next_node)
                    current[segment] = next_node
                current = next_node
            else:
                current = None
                break

        if current is None:
            continue

        leaf = path[-1]
        if leaf not in current:
            continue

        if current[leaf] != token:
            current[leaf] = token
        redacted_count += 1

    return redacted_cfg, redacted_count

### Normalizing different path inputs
We accept both dot-path strings and explicit tuples to keep call sites flexible.


In [6]:
examples = [
    "secrets.api_key",
    "  secrets.api_key  ",
    ("secrets", "api_key"),
    "strategy..credentials.passphrase",
]

for raw in examples:
    print(f"{raw!r} -> {_normalize_path(raw)}")

'secrets.api_key' -> ('secrets', 'api_key')
'  secrets.api_key  ' -> ('secrets', 'api_key')
('secrets', 'api_key') -> ('secrets', 'api_key')
'strategy..credentials.passphrase' -> ('strategy', 'credentials', 'passphrase')


### Redaction walkthrough
We copy the configuration first, then replace only the targeted leaves while leaving the original mapping untouched.


In [11]:
cfg = {
    "symbols": ["BTCUSDT"],
    "risk": {
        "max_position": 1,
        "allowed_symbols": ["BTCUSDT"],
    },
    "secrets": {
        "api_key": "demo-key-123",
        "api_secret": "super-secret",
    },
}

secret_paths = [
    "secrets.api_key",
    ("secrets", "api_secret"),
    "secrets.missing_field",  # gracefully ignored
]

redacted_cfg, redacted_count = _redact(cfg, secret_paths)

print(f"redacted_count = {redacted_count}")
print("original cfg:")
print(cfg)
print("redacted cfg:")
print(redacted_cfg)

Normalized paths:  [('secrets', 'api_key'), ('secrets', 'api_secret'), ('secrets', 'missing_field')]
traversing path: ('secrets', 'api_key')
traversing segment secrets
next_node: (current[segment]) {'api_key': 'demo-key-123', 'api_secret': 'super-secret'}
traversing path: ('secrets', 'api_secret')
traversing segment secrets
next_node: (current[segment]) {'api_key': '***REDACTED***', 'api_secret': 'super-secret'}
traversing path: ('secrets', 'missing_field')
traversing segment secrets
next_node: (current[segment]) {'api_key': '***REDACTED***', 'api_secret': '***REDACTED***'}
redacted_count = 2
original cfg:
{'symbols': ['BTCUSDT'], 'risk': {'max_position': 1, 'allowed_symbols': ['BTCUSDT']}, 'secrets': {'api_key': 'demo-key-123', 'api_secret': 'super-secret'}}
redacted cfg:
{'symbols': ['BTCUSDT'], 'risk': {'max_position': 1, 'allowed_symbols': ['BTCUSDT']}, 'secrets': {'api_key': '***REDACTED***', 'api_secret': '***REDACTED***'}}
